In [93]:
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

from pathlib import Path, PurePath
import pandas as pd
import os

In [94]:
with open("method_metrics_non_zero.txt", "r") as f:
    method_metrics_list = f.readlines()
    method_metrics_list = [x.strip() for x in method_metrics_list]

In [95]:
with open("class_metrics_non_zero.txt", "r") as f:
    class_metrics_list = f.readlines()
    class_metrics_list = [x.strip() for x in class_metrics_list]

In [98]:
df_test = pd.read_csv(f"GA_Sammon_method.csv").set_index("Repository")

new_row = {str(x): x for x in range(2, 21)}
df_test.append(pd.Series(new_row, index=df_test.columns, name='anooa'))

,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Repository,,,,,,,,,,,,,,,,,,,
aio-libs/aiohttp,524304,528400,0,20816,1578128,1118739,0,1055293,954907,0,1030929,0,1900369,1048404,1957687,2072287,0,2097111,0
aleju/imgaug,2064,524312,663568,331793,268340,264982,661785,532273,815921,163508,0,760758,1744699,1039934,1023807,0,2072319,2096887,0
ansible/ansible,524304,0,794640,528913,0,6996,817232,1711312,589329,540404,0,1014482,1023801,1810237,2079702,1023967,2031485,2097111,2031615
apache/tvm,262160,4120,1058832,12624,0,531484,547985,1708722,0,801459,2004824,2088466,1940155,1998682,654975,2023135,2096855,2031551,2097119
AUTOMATIC1111/stable-diffusion-webui,4112,528400,0,557140,276504,1578516,1063452,628376,1334806,1480219,1851185,2029201,0,2088601,1048245,2088631,0,0,2096895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XX-net/XX-Net,2064,0,1181712,0,269329,562832,21683,0,153458,1735442,0,1751734,2030196,2006591,0,2088607,2088919,2096991,2097119
yt-dlp/yt-dlp,4112,4176,17936,0,430098,1055889,268947,0,1872624,818482,1717853,2071252,1040052,1941365,1949429,2023287,2031606,0,2097119
ytdl-org/youtube-dl,24,4114,0,5328,138288,531568,0,579154,891568,1979986,0,2072338,1941086,0,0,1900255,1966039,0,2097119


In [99]:
df_test.at['aio-libs/aiohttp', '2']

524304

In [100]:
len(metric_list_non_zero)

21

In [101]:
file_data

{}

In [103]:
method = "GA"
error = "Sammon"
metric_type = "method"

df = pd.read_csv(f"{method}_{error}_{metric_type}_extra.csv").set_index("Repository")

if metric_type == "class":
    metric_list_non_zero = np.array(class_metrics_list)
elif metric_type == "method":
    metric_list_non_zero = np.array(method_metrics_list)
else:
    raise Exception()
    
#repo_list = os.listdir(f'Results/{method}_{error}/{metric_type}')
repo_list = list(Path("Results").glob(f'{method}_{error}*/{metric_type}/*'))
repo_list = [str(x).split('\\')[-1] for x in repo_list]
repo_list = [x for x in repo_list if x[-5:] == '.json']
repo_list_cleaned = [x[:-5].replace('_', '/') for x in repo_list]

repos_len = len(repo_list)
metrics_len = len(metric_list_non_zero)

out = np.zeros(shape=(len(repo_list_cleaned), metrics_len-2), dtype=np.int64)

for file_idx, file_name in enumerate(repo_list):
    if not os.path.isfile(f'Results/{method}_{error}_Method_Extra_3/{metric_type}/{file_name}'):
        continue
        
    with open(f'Results/{method}_{error}_Method_Extra_3/{metric_type}/{file_name}', 'r') as f:
        file_data = f.readline()
        file_data = json.loads(file_data)
        
    for idx, n_metrics in enumerate(range(2, len(metric_list_non_zero))):
        
        if file_name.replace('_', '/')[:-5] not in df.index:
            new_row = {str(x): 0 for x in range(2, 21)}
            df = df.append(pd.Series(new_row, index=df.columns, name=file_name.replace('_', '/')[:-5]))
        
        
        #print(df.at[file_name.replace('_', '/')[:-5], str(n_metrics)][0])
        if df.at[file_name.replace('_', '/')[:-5], str(n_metrics)] != 0:
            continue
        
        if str(n_metrics) not in file_data:
            continue
        best_n_metrics = file_data[str(n_metrics)]['feat_idx']
        best_n_metrics = metric_list_non_zero[best_n_metrics]
        
        bitmask = 0
        for metric in best_n_metrics:
            bitmask = bitmask | 2**np.where(metric_list_non_zero == metric)[0]
        
        df.at[file_name.replace('_', '/')[:-5], str(n_metrics)] = bitmask

            


df_out = df.copy()
df_out = df_out.reset_index()
df_out.to_csv(f"{method}_{error}_{metric_type}_extra.csv", index=False)
print(f"Number of NaN's: {(df_out == 0).sum().sum()}")

Number of NaN's: 9


In [104]:
df_out = df_out.reset_index()
df_out

,index,Repository,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,0,aio-libs/aiohttp,524304,528400,37392,20816,1578128,1118739,292948,1055293,...,1588948,1030929,2087760,1900369,1048404,1957687,2072287,2080734,2097111,2097119
1,1,aleju/imgaug,2064,524312,663568,331793,268340,264982,661785,532273,...,163508,1660567,760758,1744699,1039934,1023807,966623,2072319,2096887,2097119
2,2,ansible/ansible,524304,540688,794640,528913,21525,6996,817232,1711312,...,540404,898716,1014482,1023801,1810237,2079702,1023967,2031485,2097111,2031615
3,3,apache/tvm,262160,4120,1058832,12624,792080,531484,547985,1708722,...,801459,2004824,2088466,1940155,1998682,654975,2023135,2096855,2031551,2097119
4,4,AUTOMATIC1111/stable-diffusion-webui,4112,528400,534544,557140,276504,1578516,1063452,628376,...,1480219,1851185,2029201,1867697,2088601,1048245,2088631,2072567,2097111,2096895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,101,open-mmlab/mmdetection,2064,1050640,786452,1578000,300050,302097,823325,318994,...,548509,916020,1015322,2006772,1957683,917175,2080599,2096951,2080735,2097119
102,102,python-poetry/poetry,4112,557072,5392,135441,1577010,725552,1318034,832021,...,1621904,695923,1023573,1924956,2072245,1032178,1941439,1949439,2080511,2097119
103,103,ray-project/ray,262160,524306,266384,786453,692248,284945,1841936,1578396,...,1865237,1005520,2039449,1982165,1761179,1998495,1031927,2096951,2072575,2097119
104,104,sqlmapproject/sqlmap,17,1049616,671760,22552,45712,170006,105042,531410,...,112861,1613470,646774,556919,999134,1031799,2088607,2088863,2088831,2097119


In [109]:
(df_out == 0).sum()

index         1
Repository    0
2             0
3             0
4             0
5             0
6             1
7             1
8             0
9             0
10            2
11            0
12            0
13            0
14            1
15            1
16            0
17            1
18            0
19            1
20            1
dtype: int64

In [70]:
bin(37392)

'0b1001001000010000'

In [71]:
bin(528400)

'0b10000001000000010000'